In [1]:
import pickle

import pandas as pd

from sklearn.feature_extraction import DictVectorizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [2]:
from sklearn.pipeline import make_pipeline

In [3]:
import mlflow
import os

os.environ["AWS_PROFILE"] = "ArunG"
mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("green-taxi-duration")

<Experiment: artifact_location='s3://mlflow-artifacts-gansi/1', creation_time=1718859304481, experiment_id='1', last_update_time=1718859304481, lifecycle_stage='active', name='green-taxi-duration', tags={}>

In [4]:
def read_dataframe(filename: str):
    df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.dt.total_seconds() / 60
    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    return df


def prepare_dictionaries(df: pd.DataFrame):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    dicts = df[categorical + numerical].to_dict(orient='records')
    return dicts

In [5]:
df_train = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-02.parquet')

target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

dict_train = prepare_dictionaries(df_train)
dict_val = prepare_dictionaries(df_val)

In [6]:
with mlflow.start_run():
    params = dict(max_depth=20, n_estimators=100, min_samples_leaf=10, random_state=0)
    mlflow.log_params(params)

    pipeline = make_pipeline(
        DictVectorizer(),
        RandomForestRegressor(**params, n_jobs=-1)
    )

    pipeline.fit(dict_train, y_train)
    y_pred = pipeline.predict(dict_val)

    rmse = mean_squared_error(y_pred, y_val, squared=False)
    print(params, rmse)
    mlflow.log_metric('rmse', rmse)

    mlflow.sklearn.log_model(pipeline, artifact_path="model")

c:\Users\ArunGanesan\mlops-zoomcamp-garun\exp-tracking\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


{'max_depth': 20, 'n_estimators': 100, 'min_samples_leaf': 10, 'random_state': 0} 6.7558229919200725


NoCredentialsError: Unable to locate credentials

In [ ]:
from mlflow.tracking import MlflowClient


In [ ]:
MLFLOW_TRACKING_URI = 'http://127.0.0.1:5000'
RUN_ID = '8c9f23d8fd9c43d5b48a128997af5d68'

client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [ ]:
path = client.download_artifacts(run_id=RUN_ID, path='dict_vectorizer.bin')

MlflowException: The following failures occurred while downloading one or more artifacts from s3://mlflow-artifacts-gansi/4/8c9f23d8fd9c43d5b48a128997af5d68/artifacts:
##### File dict_vectorizer.bin #####
An error occurred (404) when calling the HeadObject operation: Not Found

In [ ]:
with open(path, 'rb') as f_out:
    dv = pickle.load(f_out)

In [ ]:
dv

DictVectorizer()